In [1]:
import matplotlib.pyplot as plt

from catalogues import Catalogue

for catalogue in Catalogue.all():
    matched_ra_rad, matched_dec_rad = catalogue.coords_rad(catalogue.matched_coords)
    unmatched_ra_rad, unmatched_dec_rad = catalogue.coords_rad(catalogue.unmatched_coords)
    
    plt.figure(figsize=(16,8.4))
    plt.subplot(111, projection="aitoff")
    plt.title(f'SuperWASP Crossmatch for {catalogue}')
    plt.grid(True)
    sc = plt.scatter(
        unmatched_ra_rad,
        unmatched_dec_rad,
        s=5, 
        c=catalogue.unmatched_mag,
    )
    plt.colorbar(sc)
    plt.scatter(matched_ra_rad, matched_dec_rad, s=5, c='black')
    plt.subplots_adjust(top=0.95, bottom=0.0)
    plt.show()

In [2]:
catalogues = list(Catalogue.all())
catalogues

[J/BaltA/9/646/catalog, J/ApJS/247/44/table2]

In [3]:
catalogues[0].cross_match(catalogues[1])

Matching J/BaltA/9/646/catalog against J/ApJS/247/44/table2: 2/2


_q,LC,_2MASS,f_2MASS,RAJ2000,DEJ2000,Rmag,Jmag,Hmag,Ksmag,Per,E_Per,e_Per,SNLS,FileName,Simbad,J/BaltA/9/646/catalog:GCVS,J/BaltA/9/646/catalog:RAdeg,J/BaltA/9/646/catalog:DEJ2000,J/BaltA/9/646/catalog:Per
,,,,"""h:m:s""","""d:m:s""",mag,mag,mag,mag,d,d,d,,,,,,,
int32,str2,str17,uint8,str11,str12,float32,float32,float32,float32,int16,int16,int16,float32,str67,str6,str9,float64,float64,float32
1,LC,J00240197+3834373,0,00 24 01.98,+38 34 37.38,8.40,2.02,0.77,0.12,406,440,381,4.60,KELT_N01_lc_067993_V01_east_raw_nan.dat,Simbad,R And,6.00816,38.57712,409.33
2,LC,J00222314+2659458,0,00 22 23.15,+26 59 45.83,8.69,4.68,3.92,3.50,279,293,266,5.25,KELT_N01_lc_053981_V01_east_raw_KELT_N01_lc_048462_V01_west_raw.dat,Simbad,T And,5.59647,26.99605,280.76
3,LC,J01152971+4043082,0,01 15 29.71,+40 43 08.24,8.49,4.16,3.19,2.66,340,360,322,6.84,KELT_N02_lc_061537_V01_east_raw_KELT_N02_lc_061152_V01_west_raw.dat,Simbad,U And,18.87225,40.71956,346.55
4,LC,J00500630+3539098,0,00 50 06.31,+35 39 09.89,9.36,5.48,4.69,4.27,260,271,250,5.41,KELT_N01_lc_041955_V01_east_raw_KELT_N01_lc_030143_V01_west_raw.dat,Simbad,V And,12.52481,35.65333,257.73
5,LC,J02173296+4418176,0,02 17 32.96,+44 18 17.70,7.39,1.59,0.51,0.13,395,434,366,5.33,nan_KELT_N02_lc_063557_V01_west_raw.dat,Simbad,W And,34.38734,44.30494,395.93
6,LC,J00160960+4700444,0,00 16 09.60,+47 00 44.48,7.83,4.86,3.60,2.97,345,385,310,3.58,KELT_N16_lc_021497_V01_east_raw_KELT_N16_lc_001236_V01_west_raw.dat,Simbad,X And,4.03768,47.01267,346.18
7,LC,J01393689+3920348,0,01 39 36.89,+39 20 34.88,9.72,5.25,4.42,4.15,223,232,214,5.39,KELT_N02_lc_051192_V01_east_raw_KELT_N02_lc_064008_V01_west_raw.dat,Simbad,Y And,24.9031,39.34409,220.53
8,LC,J00512328+3422363,0,00 51 23.29,+34 22 36.32,10.89,4.88,3.84,3.25,329,351,311,4.45,KELT_N01_lc_047788_V01_east_raw_nan.dat,Simbad,RR And,12.84605,34.37718,328.15
